# Benchmark EDUC at Large Scale

This notebook aims to benchmark the economic dispatch and unit commitment problems at large scale.

In [1]:
import pandas as pd

import ams

from andes.utils.misc import elapsed

ams.config_logger(stream_level=50)

%run ../benchmarks.py
_ = get_tool_versions()

Package mosek not found.


Last run time: 2025-02-02 00:16:24
Python: 3.12.0 | packaged by conda-forge | (main, Oct  3 2023, 08:36:57) [Clang 15.0.7 ]

Tool        Version      
----------  -------------
ltbams      1.0.2        
andes       1.9.3        
cvxpy       1.6.0        
gurobipy    12.0.1       
mosek       Not installed
piqp        0.4.2        
pandapower  2.14.7       
numba       0.60.0       


In [2]:
def test_educ(case, routine='UC', solver='GUROBI', ignore_dpp=True, ):
    """
    Test the execution time of the specified routine on the given case.

    Parameters
    ----------
    case : str
        The path to the case file.
    routine : str, optional
        The name of the routine to test. Defaults to 'DCOPF'.
    solver : str, optional
        The name of the solver to use. Defaults to 'GUROBI'
    ignore_dpp : bool, optional
        Whether to ignore DPP. Defaults to True.

    Returns
    -------
    tuple
        A tuple containing the list of times and the list of objective values.
    """
    sp = ams.load(case, setup=True, default_config=True, no_output=True)

    rtn = sp.routines[routine]

    # Initialize AMS
    # --- matrices build ---
    t_mats, _ = elapsed()
    sp.mats.build()
    _, s_mats = elapsed(t_mats)

    # --- code generation ---
    t_parse, _ = elapsed()
    rtn.om.parse()
    _, s_parse = elapsed(t_parse)

    # --- code evaluation ---
    t_evaluate, _ = elapsed()
    rtn.om.evaluate()
    _, s_evaluate = elapsed(t_evaluate)

    # --- problem finalization ---
    t_finalize, _ = elapsed()
    rtn.om.finalize()
    _, s_finalize = elapsed(t_finalize)

    # --- rest init process ---
    t_postinit, _ = elapsed()
    rtn.init()
    _, s_postinit = elapsed(t_postinit)

    # --- run solvers ---
    t_ams_solve, _ = elapsed()
    rtn.run(solver=solver, ignore_dpp=ignore_dpp)
    _, s_ams_solve = elapsed(t_ams_solve)

    time = [s_mats, s_parse, s_evaluate, s_finalize,
            s_postinit, s_ams_solve]
    time = [float(t.split(' ')[0]) for t in time]

    return time

In [3]:
# run a small case to ensure test is working
_case0 = ams.get_case('ieee14/ieee14_uced.xlsx')
test_educ(_case0, routine='ED', solver='GUROBI', ignore_dpp=True)

Set parameter Username
Set parameter LicenseID to value 2617183
Academic license - for non-commercial use only - expires 2026-02-02


[0.0006, 0.0049, 0.0067, 0.0001, 0.0, 0.0301]

In [4]:
case = '../UCCase/goc31777_uced.xlsx'
t_ed = test_educ(case, routine='ED', solver='GUROBI', ignore_dpp=True)

In [5]:
case = '../UCCase/goc31777_uced.xlsx'
t_uc = test_educ(case, routine='UC', solver='GUROBI', ignore_dpp=True)

In [6]:
# Define the column names and the results
cols = ['ams_mats', 'ams_parse', 'ams_eval',
        'ams_final', 'ams_postinit', 'ams_solver']

# Create a DataFrame
df = pd.DataFrame({'time': cols, 'UC': t_uc, 'ED': t_ed})

df

,time,UC,ED
0,ams_mats,0.2083,0.2182
1,ams_parse,0.0057,0.0039
2,ams_eval,41.7867,52.9578
3,ams_final,0.0001,0.0001
4,ams_postinit,0.0006,0.0000
5,ams_solver,42.3423,21.0474


In [7]:
df.to_csv('../results/results_educ_large.csv')